In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("andrewmvd/sp-500-stocks")

print("Path to dataset files:", path)

/Users/leonidalekseev/VSCodeProjects/sp500-d3-graph/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


100%|██████████| 44.7M/44.7M [00:04<00:00, 10.3MB/s]

Extracting files...


Path to dataset files: /Users/leonidalekseev/.cache/kagglehub/datasets/andrewmvd/sp-500-stocks/versions/1017


In [1]:
import os
import pandas as pd

# Создание DataFrame
path = "/Users/leonidalekseev/.cache/kagglehub/datasets/andrewmvd/sp-500-stocks/versions/1017"
df = pd.read_csv(os.path.join(path, "sp500_stocks.csv"))
df_info = pd.read_csv(os.path.join(path, "sp500_companies.csv"))

In [8]:
df.head()

,Date,Symbol,Adj Close,Close,High,Low,Open,Volume
0,2010-01-04,MMM,46.422302,69.414719,69.774246,69.122070,69.473244,3640265.0
1,2010-01-05,MMM,46.131523,68.979935,69.590302,68.311035,69.230766,3405012.0
2,2010-01-06,MMM,46.785759,69.958191,70.735786,69.824417,70.133781,6301126.0
3,2010-01-07,MMM,46.819294,70.008362,70.033447,68.662209,69.665550,5346240.0
4,2010-01-08,MMM,47.149204,70.501671,70.501671,69.648827,69.974915,4073337.0


In [7]:
df_info.head()

,Exchange,Symbol,Shortname,Longname,Sector,Industry,Currentprice,Marketcap,Ebitda,Revenuegrowth,City,State,Country,Fulltimeemployees,Longbusinesssummary,Weight
0,NMS,AAPL,Apple Inc.,Apple Inc.,Technology,Consumer Electronics,251.04,3794670190592,1.346610e+11,0.061,Cupertino,CA,United States,164000.0,"Apple Inc. designs, manufactures, and markets ...",0.066601
1,NMS,MSFT,Microsoft Corporation,Microsoft Corporation,Technology,Software - Infrastructure,451.59,3357517545472,1.365520e+11,0.160,Redmond,WA,United States,228000.0,Microsoft Corporation develops and supports so...,0.058928
2,NMS,NVDA,NVIDIA Corporation,NVIDIA Corporation,Technology,Semiconductors,132.00,3232680116224,6.118400e+10,1.224,Santa Clara,CA,United States,29600.0,NVIDIA Corporation provides graphics and compu...,0.056737
3,NMS,AMZN,"Amazon.com, Inc.","Amazon.com, Inc.",Consumer Cyclical,Internet Retail,232.93,2449258840064,1.115830e+11,0.110,Seattle,WA,United States,1551000.0,"Amazon.com, Inc. engages in the retail sale of...",0.042987
4,NMS,GOOGL,Alphabet Inc.,Alphabet Inc.,Communication Services,Internet Content & Information,196.66,2416263036928,1.234700e+11,0.151,Mountain View,CA,United States,181269.0,Alphabet Inc. offers various products and plat...,0.042408


In [2]:
# Преобразование столбца 'Date' в datetime
df["Date"] = pd.to_datetime(df["Date"])

# Добавление года в отдельный столбец
df["Year"] = df["Date"].dt.year

# Группировка данных по году и символу
result = (
    df.groupby(["Year", "Symbol"])
    .agg(
        lastPrice=("Close", "last"),
        meanPrice=("Close", "mean"),
        minPrice=("Low", "min"),
        maxPrice=("High", "max"),
        totalVolume=("Volume", "sum"),
    )
    .reset_index()
)

# Получение нужной информации
sectors_unique = df_info.Sector.unique().tolist()

In [3]:
# Преобразование в нужный формат
node_result = {}
for _, row in result.iterrows():
    year = row["Year"]
    if year not in node_result:
        node_result[year] = []

    symbol = row["Symbol"]
    name = df_info[df_info.Symbol == symbol]["Longname"].tolist()[0]
    sector = df_info[df_info.Symbol == symbol]["Sector"].tolist()[0]
    sector_id = sectors_unique.index(sector)
    node_result[year].append(
        {
            "id": symbol,
            "name": name,
            "sector": sector,
            "sectorId": sector_id,
            "lastPrice": row["lastPrice"],
            "meanPrice": row["meanPrice"],
            "minPrice": row["minPrice"],
            "maxPrice": row["maxPrice"],
            "totalVolume": row["totalVolume"],
        }
    )

In [4]:
# Пивотирование данных для получения цен закрытия в формате, удобном для расчета корреляции
pivot_df = df.pivot(index="Date", columns="Symbol", values="Close")

# Преобразование в нужный формат
edge_results = {}
for year in pivot_df.index.year.unique():
    yearly_data = pivot_df[pivot_df.index.year == year]

    # Вычисление корреляции
    correlation_matrix = yearly_data.corr()

    # Сбор пар акций и их корреляции
    correlations = []
    for i in range(len(correlation_matrix.columns)):
        for j in range(i):
            source = correlation_matrix.columns[i]
            target = correlation_matrix.columns[j]
            correlation_value = float(correlation_matrix.iloc[i, j])
            if correlation_value > 0.5:
                correlations.append(
                    {
                        "source": source,
                        "target": target,
                        "correlation": correlation_value,
                    }
                )

    edge_results[year] = correlations

In [52]:
node_result[2024][:2]

[{'id': 'A',
  'name': 'Agilent Technologies, Inc.',
  'sector': 'Healthcare',
  'sectorId': 5,
  'lastPrice': 137.6699981689453,
  'meanPrice': 137.79876009097768,
  'minPrice': 124.16000366210938,
  'maxPrice': 155.35000610351562,
  'totalVolume': 416595827.0},
 {'id': 'AAPL',
  'name': 'Apple Inc.',
  'sector': 'Technology',
  'sectorId': 0,
  'lastPrice': 251.0399932861328,
  'meanPrice': 205.28669398284154,
  'minPrice': 164.0800018310547,
  'maxPrice': 251.37890625,
  'totalVolume': 13869845677.0}]

In [54]:
edge_results[2024][:2]

[{'source': 'AAPL', 'target': 'A', 'correlation': -0.1942282767501067},
 {'source': 'ABBV', 'target': 'A', 'correlation': 0.20831439671156127}]

In [5]:
# Фильтруем акции которые не существовали в определенные годы
def filer_results(results):
    filtered_results = {}
    for year, data in results.items():
        filtered_data = []
        for d in data:
            if all([pd.notna(v) for v in d.values()]):
                filtered_data.append(d)
        filtered_results[year] = filtered_data

    return filtered_results


node_result = filer_results(node_result)
edge_results = filer_results(edge_results)

In [6]:
import json

# Путь к папке
output_dir = "../public/data"
os.makedirs(output_dir, exist_ok=True)

# Сохранение результатов в JSON файл
output_file = os.path.join(output_dir, "node_result.json")
with open(output_file, "w") as f:
    json.dump(node_result, f, indent=4)

print(f"Результаты сохранены в {output_file}")

# Сохранение результатов в JSON файл
output_file = os.path.join(output_dir, "edge_results.json")
with open(output_file, "w") as f:
    json.dump(edge_results, f, indent=4)

print(f"Результаты сохранены в {output_file}")

Результаты сохранены в ../public/data/node_result.json
Результаты сохранены в ../public/data/edge_results.json
